# Let's write the code

<img src="http://j.mp/1VAmanI">

We can now launch the stack!

- The init phase is needed only once. 
- As soon as you keep the initialized volume, everything will stay in place with persistence.

```bash
# Launch
scripts/run.sh
```

To open the bash shell for you flask python server, use the command:

```bash
scripts/run.sh shell_server
```

To further get another container shell for testing the APIs:

```bash
scripts/run.sh shell_client
```

# Managing your stack

```bash
# Stop the whole stack
scripts/run.sh stop
```

- After this command containers are freezed but still restartable
- Even if you rebooth the host machine

```bash
# Destroy containers (you lose unsaved data)
scripts/run.sh remove
```

```bash

# Start containers by removing existing (stop -> remove -> start)
scripts/run.sh

# Start containers by only restarting what is down
scripts/run.sh graceful
```

## How to destroy all containers (!)


```bash
docker rm -f $(docker ps -a -q)
```

## How to destroy all volumes (!!)


```bash
docker volume rm $(docker volume ls -q)
```

# Steps to use the `http-api` library

1) Create a python class from a template

In [ ]:
%%bash

# From the root directory of the github repository

cp vanilla/apis/foo.py vanilla/apis/mynewendpoint.py

vim vanilla/apis/mynewendpoint.py

In [ ]:
%%writefile vanilla/apis/mynewendpoint.py

from ..base import ExtendedApiResource
from .. import decorators as decorate

from restapi import get_logger
logger = get_logger(__name__)

# 1. The class must inherit ExtendedApiResource

class SomeRestEndpoint(ExtendedApiResource):
    """ My first python endpoint """

# 2. Only the REST API methods must be decorated as 'apimethod'
# e.g. GET, POST, PUT, DELETE and so on
    
    @decorate.apimethod
    def get(self):
        hello = "Hello world"
        logger.info(hello)
        return self.response(hello)


2) Define a JSON init file


```bash
# example of content
cat vanilla/specs/api_init.json

{ "EUDAT http api": "eudat.ini" }
```

3) Configure classes to your desired endpoints urls

```bash
cat vanilla/specs/eudat.ini

{
    "custom.mynewendpoint": {
        "SomeRestEndpoint": [
            "helloworld"
        ]
    }
}

```

With this configuration your `get` method will be available at the url

http://localhost:8080/api/helloworld

4) Startup the Flask server

```bash
scripts/run.sh shell_server

$ ./boot
```

### Possible problems experienced so far

1) No graphdb connection available, and Flask exits
    - neo4j startup may take seconds
    - You may check any status with `docker-compose logs SERVICE`
    - so you may try `docker-compose logs graphdb`


2) No irods connection available, and Flask exits
    - irods validate its json configuration files against its website
    - you most likely have irods down if you work offline


5) Connect to resources

# API accounts

The base of our Flask API provide a simple accounting.
This should be expanded with our Authentication schema.

You may use a dummy endpoint to verify if your user is logged:

```bash

$ http GET http://api:5000/api/verifylogged


HTTP/1.0 401 UNAUTHORIZED
Content-Length: 271
Content-Type: text/html; charset=utf-8
Date: Tue, 23 Feb 2016 22:59:56 GMT
Server: Werkzeug/0.11.4 Python/3.4.3+

    <h1>Unauthorized</h1>
    <p>The server could not verify that you are authorized to access the URL
    requested. You either supplied the wrong credentials (e.g. a bad password),
    or your browser doesn't understand how to supply the credentials required.</p>
    
```

At first the backend is connected to a sqllite database located in `/dbs/backend.db` inside the `rest` container.

Dummy roles already exists after the init phase:
- adminer
- justauser

Only one dummy user is provided as administrator:
- user@nomail.org

To test the login phase you may user the `/login` endpoint

```bash
http POST http://api:5000/api/login email=user@nomail.org password=test
```

Response would be something like:
    
```bash
HTTP/1.0 200 OK
Content-Length: 215
Content-Type: application/json
Date: Tue, 23 Feb 2016 23:00:40 GMT
Server: Werkzeug/0.11.4 Python/3.4.3+
Set-Cookie: session=.eJwljksKwzAMBe-idRb62Y5ymWLLEu02aVald6-hvM3AMPA-8Mgzricc7_OODR6vCQdM9QyyNhzdhSiL-1yE6hJdNGTJms0RXdoo3FSFkDls9F3MOmGVteTV2W4zXbspVQ8caFlwzFbdOQtV5bTiNpEbp_auEza4rzj_Zwi-P7pBLyg.Ca52GA.gDEtwn3TOMAbYVysR0rQxeKtiTE; HttpOnly; Path=/

{
    "meta": {
        "code": 200
    },
    "response": {
        "user": {
            "authentication_token": "WyIxIiwiYmEwNzZjMmU0MGE0YmZlMWQ4N2JjNmEzYTZmNzE5NTMiXQ.Ca52GA.SzIk0KjuD7Pb0bhjnDk7caufdXo",
            "id": "1"
        }
    }
}
```

Now you can use the token to access an endpoint which has restricted permissions:

```bash
http GET http://api:5000/api/verifylogged \
    Authentication-token:WyIxIiwiYmEwNzZjMmU0MGE0YmZlMWQ4N2JjNmEzYTZmNzE5NTMiXQ.Ca52GA.SzIk0KjuD7Pb0bhjnDk7caufdXo


HTTP/1.0 200 OK
Content-Length: 48
Content-Type: application/json
Date: Tue, 23 Feb 2016 23:01:34 GMT
Server: Werkzeug/0.11.4 Python/3.4.3+

{
    "data": null,
    "data_type": null,
    "elements": 0
}
```

Finally, we have a last dummy endpoint to verify admin privileges:

``` bash
http GET http://api:5000/api/verifyadmin
```

# Restricting your endpoint

To let only logged user which provide a valid token, access your endpoint,
you need the `auth_token_required` decorator:

```python
from flask.ext.security import auth_token_required

class MyEndpoint(ExtendedApiResource):

    @decorate.apimethod
    @auth_token_required
    def get(self):
        return self.response("Only if logged")
```

To specify which roles are needed (one or more), there is another decorator called
`roles_required`:

```python
from flask.ext.security import roles_required, auth_token_required
from confs import config

class MyEndpoint(ExtendedApiResource):

    @decorate.apimethod
    @auth_token_required
    @roles_required(config.ROLE_ADMIN)
    def get(self):
        return self.response("Only if admin")
```

# Managing users

You can define more roles inside `confs/config.py`.

To manage users and roles 
(saved inside the db) 
by WUI 
we can use the already available Flask-Admin plugin.

The url endpoint (to access via browser) is `/api/manage`.

Of course it will require a valid `adminer` role:
    
http://localhost:8080/api/manage

<img src="http://memeshappen.com/media/created/finally-We-did-it-meme-6891.jpg">